# Oracle Database Memory Store for AI Agents

This notebook demonstrates a production-ready memory store using Oracle Database for AI agents with LangChain integration.

## Features
- **LangChain SQLChatMessageHistory** for conversation persistence
- **LangChain SQL Database agent** for querying
- **Vector embeddings** with Oracle Vector Store
- **Full LangChain memory integration**
- Entity tracking and session management
- Task execution records
- Agent state management

## Overview
This implementation provides a robust memory system for AI agents using Oracle Database as the backend, enabling:
- Persistent conversation history
- Entity extraction and tracking
- Task execution records
- Vector embeddings for semantic search
- Integration with LangChain agents and chains

## 1. Install Required Dependencies

Run the cell below to install all required packages for Google Colab environment.

In [ ]:
%%capture
import subprocess
import sys

# Install required packages for Google Colab
packages_to_install = [
    "sqlalchemy>=2.0.0",
    "oracledb>=1.4.0",
    "langchain>=0.0.300",
    "langchain-community>=0.0.10",
    "python-dotenv>=1.0.0"
]

print("Installing required packages...")
for package in packages_to_install:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])
    
print("✓ All packages installed successfully!")

## 2. Import Libraries and Setup

In [ ]:
import json
from datetime import datetime
from typing import Dict, List, Optional, Any, Tuple
import sqlalchemy as sa
from sqlalchemy import create_engine, Column, String, Text, DateTime, Integer, Index, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, Session

from langchain.memory import BaseMemory, ConversationBufferMemory, ConversationSummaryMemory
from langchain.chat_history import BaseChatMessageHistory
from langchain.schema import BaseMessage, HumanMessage, AIMessage
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.llms.base import BaseLanguageModel
from langchain.embeddings.base import Embeddings

import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import os

print("✓ All imports successful!")

## 3. Define Oracle Database Models

SQLAlchemy ORM models for storing:
- **ConversationMemory**: Agent conversation history
- **EntityMemory**: Extracted entities and facts
- **AgentTask**: Task execution records
- **VectorEmbedding**: Vector embeddings for semantic search

In [ ]:
# ============================================================================
# Oracle Database Models (SQLAlchemy ORM)
# ============================================================================

Base = declarative_base()


class ConversationMemory(Base):
    """Stores conversation history in Oracle database"""
    __tablename__ = "agent_conversation_memory"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    session_id = Column(String(255), nullable=False, index=True)
    agent_id = Column(String(255), nullable=False, index=True)
    message_type = Column(String(50), nullable=False)  # 'human' or 'ai'
    content = Column(Text, nullable=False)
    metadata = Column(Text)  # JSON string
    created_at = Column(DateTime, default=datetime.utcnow, index=True)
    
    __table_args__ = (
        Index('idx_session_agent_created', 'session_id', 'agent_id', 'created_at'),
    )
    
    def to_message(self) -> BaseMessage:
        """Convert DB record to LangChain message"""
        if self.message_type == "human":
            return HumanMessage(content=self.content)
        else:
            return AIMessage(content=self.content)


class EntityMemory(Base):
    """Stores extracted entities and facts for agent reasoning"""
    __tablename__ = "agent_entity_memory"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    session_id = Column(String(255), nullable=False, index=True)
    agent_id = Column(String(255), nullable=False, index=True)
    entity_type = Column(String(100), nullable=False)  # 'person', 'location', etc.
    entity_name = Column(String(255), nullable=False)
    entity_value = Column(Text, nullable=False)  # JSON string for complex data
    last_mentioned = Column(DateTime, default=datetime.utcnow)
    mention_count = Column(Integer, default=1)
    
    __table_args__ = (
        Index('idx_session_entity_type', 'session_id', 'agent_id', 'entity_type'),
        Index('idx_entity_name', 'entity_name'),
    )


class AgentTask(Base):
    """Stores agent tasks and execution history"""
    __tablename__ = "agent_tasks"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    session_id = Column(String(255), nullable=False, index=True)
    agent_id = Column(String(255), nullable=False, index=True)
    task_description = Column(Text, nullable=False)
    status = Column(String(50), default="pending")  # pending, in_progress, completed, failed
    result = Column(Text)  # JSON string
    created_at = Column(DateTime, default=datetime.utcnow)
    completed_at = Column(DateTime)
    
    __table_args__ = (
        Index('idx_session_status', 'session_id', 'status'),
    )


class VectorEmbedding(Base):
    """Stores vector embeddings for semantic search"""
    __tablename__ = "agent_vector_embeddings"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    session_id = Column(String(255), nullable=False, index=True)
    agent_id = Column(String(255), nullable=False, index=True)
    content = Column(Text, nullable=False)
    embedding = Column(Text, nullable=False)  # JSON array as string
    metadata = Column(Text)  # JSON string with source, type, etc.
    created_at = Column(DateTime, default=datetime.utcnow)
    
    __table_args__ = (
        Index('idx_session_vector', 'session_id', 'agent_id'),
    )

print("✓ Database models defined successfully!")

## 4. LangChain Chat Message History Implementation

Define the LangChain-compatible chat message history using Oracle database backend.

In [ ]:
# ============================================================================
# LangChain Chat Message History
# ============================================================================

class OracleSQLChatMessageHistory(BaseChatMessageHistory):
    """
    LangChain-compatible chat message history using Oracle database.
    Implements the BaseChatMessageHistory interface from LangChain.
    """
    
    def __init__(
        self,
        engine: sa.engine.Engine,
        session_id: str,
        agent_id: str = "default_agent",
    ):
        """Initialize Oracle chat message history"""
        self.engine = engine
        self.session_id = session_id
        self.agent_id = agent_id
        self.SessionLocal = sessionmaker(bind=engine)
    
    @property
    def messages(self) -> List[BaseMessage]:
        """Get all messages from Oracle"""
        session = self.SessionLocal()
        try:
            records = session.query(ConversationMemory).filter(
                ConversationMemory.session_id == self.session_id,
                ConversationMemory.agent_id == self.agent_id,
            ).order_by(ConversationMemory.created_at).all()
            
            return [msg.to_message() for msg in records]
        finally:
            session.close()
    
    def add_message(self, message: BaseMessage) -> None:
        """Add a message to Oracle"""
        session = self.SessionLocal()
        try:
            if isinstance(message, HumanMessage):
                msg_type = "human"
            elif isinstance(message, AIMessage):
                msg_type = "ai"
            else:
                msg_type = "system"
            
            db_msg = ConversationMemory(
                session_id=self.session_id,
                agent_id=self.agent_id,
                message_type=msg_type,
                content=message.content,
            )
            session.add(db_msg)
            session.commit()
        finally:
            session.close()
    
    def clear(self) -> None:
        """Clear all messages for this session"""
        session = self.SessionLocal()
        try:
            session.query(ConversationMemory).filter(
                ConversationMemory.session_id == self.session_id,
                ConversationMemory.agent_id == self.agent_id,
            ).delete()
            session.commit()
        finally:
            session.close()

print("✓ LangChain Chat Message History defined!")

## 5. Oracle Connection Configuration

Define configuration class for Oracle database connection with support for both service names and SIDs.

In [ ]:
# ============================================================================
# Oracle Connection Configuration
# ============================================================================

class OracleMemoryConfig:
    """Configuration for Oracle memory store and LangChain integration"""
    
    def __init__(
        self,
        username: str,
        password: str,
        host: str,
        port: int = 1521,
        service_name: Optional[str] = None,
        sid: Optional[str] = None,
    ):
        """
        Initialize Oracle connection configuration.
        
        Args:
            username: Oracle username
            password: Oracle password
            host: Oracle host address
            port: Oracle port (default 1521)
            service_name: Oracle service name (for Oracle Cloud/modern setup)
            sid: Oracle SID (for on-premise traditional setup)
        """
        self.username = username
        self.password = password
        self.host = host
        self.port = port
        self.service_name = service_name
        self.sid = sid
        
        if not service_name and not sid:
            raise ValueError("Either service_name or sid must be provided")
    
    def get_connection_string(self) -> str:
        """Generate Oracle connection string for SQLAlchemy"""
        if self.service_name:
            # Oracle Cloud or modern setup
            dsn = f"{self.host}:{self.port}/{self.service_name}"
            return f"oracle+oracledb://{self.username}:{self.password}@{dsn}"
        else:
            # Traditional setup with SID
            dsn = f"{self.host}:{self.port}:{self.sid}"
            return f"oracle+oracledb://{self.username}:{self.password}@{dsn}"

print("✓ Oracle Configuration class defined!")

## 6. Oracle Agent Memory Implementation

Complete implementation of LangChain-compatible memory store using Oracle database with full support for conversation history, entity management, and vector embeddings.

In [ ]:
# ============================================================================
# Oracle Memory Store Implementation
# ============================================================================

class OracleAgentMemory(BaseMemory):
    """
    LangChain-compatible memory store using Oracle database.
    
    Provides persistent storage for:
    - Conversation history
    - Entity extraction and tracking
    - Task execution records
    - Agent state management
    """
    
    def __init__(
        self,
        engine: sa.engine.Engine,
        session_id: str,
        agent_id: str = "default_agent",
        max_context_length: int = 10,
    ):
        """
        Initialize Oracle agent memory.
        
        Args:
            engine: SQLAlchemy engine for Oracle
            session_id: Unique conversation session ID
            agent_id: Agent identifier
            max_context_length: Max messages to retrieve for context window
        """
        super().__init__()
        self.engine = engine
        self.session_id = session_id
        self.agent_id = agent_id
        self.max_context_length = max_context_length
        self.SessionLocal = sessionmaker(bind=engine)
    
    @property
    def memory_variables(self) -> List[str]:
        """Return memory variable names for LangChain"""
        return ["history"]
    
    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Retrieve conversation history from Oracle"""
        session = self.SessionLocal()
        try:
            messages = session.query(ConversationMemory).filter(
                ConversationMemory.session_id == self.session_id,
                ConversationMemory.agent_id == self.agent_id,
            ).order_by(ConversationMemory.created_at).limit(
                self.max_context_length
            ).all()
            
            history = "\n".join([
                f"{msg.message_type.upper()}: {msg.content}" 
                for msg in messages
            ])
            return {"history": history}
        finally:
            session.close()
    
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save user input and AI response to Oracle"""
        session = self.SessionLocal()
        try:
            # Save user message
            if "input" in inputs:
                user_msg = ConversationMemory(
                    session_id=self.session_id,
                    agent_id=self.agent_id,
                    message_type="human",
                    content=inputs["input"],
                    metadata=json.dumps({"type": "user_input"})
                )
                session.add(user_msg)
            
            # Save AI response
            if "output" in outputs:
                ai_msg = ConversationMemory(
                    session_id=self.session_id,
                    agent_id=self.agent_id,
                    message_type="ai",
                    content=outputs["output"],
                    metadata=json.dumps({"type": "agent_response"})
                )
                session.add(ai_msg)
            
            session.commit()
        except Exception as e:
            session.rollback()
            raise Exception(f"Error saving context to Oracle: {e}")
        finally:
            session.close()
    
    def clear(self) -> None:
        """Clear all conversation history for this session"""
        session = self.SessionLocal()
        try:
            session.query(ConversationMemory).filter(
                ConversationMemory.session_id == self.session_id,
                ConversationMemory.agent_id == self.agent_id,
            ).delete()
            session.commit()
        finally:
            session.close()
    
    def add_entity(self, entity_type: str, entity_name: str, entity_value: Dict) -> None:
        """Store extracted entity in Oracle"""
        session = self.SessionLocal()
        try:
            existing = session.query(EntityMemory).filter(
                EntityMemory.session_id == self.session_id,
                EntityMemory.agent_id == self.agent_id,
                EntityMemory.entity_type == entity_type,
                EntityMemory.entity_name == entity_name,
            ).first()
            
            if existing:
                existing.entity_value = json.dumps(entity_value)
                existing.last_mentioned = datetime.utcnow()
                existing.mention_count += 1
            else:
                entity = EntityMemory(
                    session_id=self.session_id,
                    agent_id=self.agent_id,
                    entity_type=entity_type,
                    entity_name=entity_name,
                    entity_value=json.dumps(entity_value),
                )
                session.add(entity)
            
            session.commit()
        finally:
            session.close()
    
    def get_entities(self, entity_type: Optional[str] = None) -> List[Dict]:
        """Retrieve stored entities from Oracle"""
        session = self.SessionLocal()
        try:
            query = session.query(EntityMemory).filter(
                EntityMemory.session_id == self.session_id,
                EntityMemory.agent_id == self.agent_id,
            )
            
            if entity_type:
                query = query.filter(EntityMemory.entity_type == entity_type)
            
            entities = []
            for e in query.all():
                entities.append({
                    "type": e.entity_type,
                    "name": e.entity_name,
                    "value": json.loads(e.entity_value),
                    "mention_count": e.mention_count,
                    "last_mentioned": e.last_mentioned.isoformat(),
                })
            
            return entities
        finally:
            session.close()
    
    def add_task(self, task_description: str, result: Optional[Dict] = None) -> int:
        """Record a task or action taken by the agent"""
        session = self.SessionLocal()
        try:
            task = AgentTask(
                session_id=self.session_id,
                agent_id=self.agent_id,
                task_description=task_description,
                status="in_progress",
                result=json.dumps(result) if result else None,
            )
            session.add(task)
            session.commit()
            return task.id
        finally:
            session.close()
    
    def update_task_status(self, task_id: int, status: str, result: Optional[Dict] = None) -> None:
        """Update task status and result"""
        session = self.SessionLocal()
        try:
            task = session.query(AgentTask).filter(AgentTask.id == task_id).first()
            if task:
                task.status = status
                if result:
                    task.result = json.dumps(result)
                if status == "completed":
                    task.completed_at = datetime.utcnow()
                session.commit()
        finally:
            session.close()
    
    def get_conversation_history(self, limit: Optional[int] = None) -> List[Dict]:
        """Get complete conversation history for this session"""
        session = self.SessionLocal()
        try:
            query = session.query(ConversationMemory).filter(
                ConversationMemory.session_id == self.session_id,
                ConversationMemory.agent_id == self.agent_id,
            ).order_by(ConversationMemory.created_at)
            
            if limit:
                query = query.limit(limit)
            
            history = []
            for msg in query.all():
                history.append({
                    "id": msg.id,
                    "type": msg.message_type,
                    "content": msg.content,
                    "created_at": msg.created_at.isoformat(),
                    "metadata": json.loads(msg.metadata) if msg.metadata else {},
                })
            
            return history
        finally:
            session.close()
    
    def get_chat_message_history(self) -> OracleSQLChatMessageHistory:
        """Get LangChain-compatible chat message history"""
        return OracleSQLChatMessageHistory(
            engine=self.engine,
            session_id=self.session_id,
            agent_id=self.agent_id
        )
    
    def add_embedding(self, content: str, embedding: List[float], metadata: Optional[Dict] = None) -> None:
        """Store vector embedding for semantic search"""
        session = self.SessionLocal()
        try:
            vector_record = VectorEmbedding(
                session_id=self.session_id,
                agent_id=self.agent_id,
                content=content,
                embedding=json.dumps(embedding),
                metadata=json.dumps(metadata) if metadata else None,
            )
            session.add(vector_record)
            session.commit()
        finally:
            session.close()
    
    def get_embeddings(self, limit: Optional[int] = None) -> List[Dict]:
        """Retrieve stored embeddings for semantic search"""
        session = self.SessionLocal()
        try:
            query = session.query(VectorEmbedding).filter(
                VectorEmbedding.session_id == self.session_id,
                VectorEmbedding.agent_id == self.agent_id,
            ).order_by(VectorEmbedding.created_at)
            
            if limit:
                query = query.limit(limit)
            
            embeddings = []
            for record in query.all():
                embeddings.append({
                    "id": record.id,
                    "content": record.content,
                    "embedding": json.loads(record.embedding),
                    "metadata": json.loads(record.metadata) if record.metadata else {},
                    "created_at": record.created_at.isoformat(),
                })
            
            return embeddings
        finally:
            session.close()

print("✓ Oracle Agent Memory class defined!")

## 7. LangChain Agent Builders

Helper classes for creating LangChain SQL agents and memory chains with Oracle backend.

In [ ]:
# ============================================================================
# LangChain Agent Builders
# ============================================================================

class OracleSQLAgentBuilder:
    """Builder for creating LangChain SQL agents with Oracle database"""
    
    @staticmethod
    def create_sql_agent(
        oracle_config: OracleMemoryConfig,
        llm: BaseLanguageModel,
        verbose: bool = True,
    ):
        """Create a LangChain SQL agent for Oracle database queries"""
        engine = create_engine(oracle_config.get_connection_string())
        db = SQLDatabase(engine)
        toolkit = SQLDatabaseToolkit(db=db, llm=llm)
        
        agent = create_sql_agent(
            llm=llm,
            toolkit=toolkit,
            verbose=verbose,
            agent_type="openai-tools",
        )
        
        return agent


class OracleLangChainMemoryBuilder:
    """Builder for creating LangChain memory chains with Oracle"""
    
    @staticmethod
    def create_conversation_memory(
        oracle_config: OracleMemoryConfig,
        session_id: str,
        agent_id: str = "default_agent",
        llm: Optional[BaseLanguageModel] = None,
    ) -> ConversationBufferMemory:
        """Create LangChain ConversationBufferMemory with Oracle backend"""
        memory = create_oracle_agent_memory(
            oracle_config=oracle_config,
            session_id=session_id,
            agent_id=agent_id
        )
        
        return ConversationBufferMemory(
            chat_memory=memory.get_chat_message_history(),
            return_messages=True
        )
    
    @staticmethod
    def create_conversation_summary_memory(
        oracle_config: OracleMemoryConfig,
        session_id: str,
        llm: BaseLanguageModel,
        agent_id: str = "default_agent",
    ) -> ConversationSummaryMemory:
        """Create LangChain ConversationSummaryMemory with Oracle backend"""
        memory = create_oracle_agent_memory(
            oracle_config=oracle_config,
            session_id=session_id,
            agent_id=agent_id
        )
        
        return ConversationSummaryMemory(
            llm=llm,
            chat_memory=memory.get_chat_message_history(),
            return_messages=True
        )

print("✓ LangChain Agent Builders defined!")

## 8. Factory and Utility Functions

Helper functions for creating and testing Oracle memory stores.

In [ ]:
# ============================================================================
# Factory and Utility Functions
# ============================================================================

def create_oracle_agent_memory(
    oracle_config: OracleMemoryConfig,
    session_id: str,
    agent_id: str = "default_agent",
    create_tables: bool = True
) -> OracleAgentMemory:
    """Factory function to create Oracle memory store"""
    engine = create_engine(oracle_config.get_connection_string())
    
    if create_tables:
        Base.metadata.create_all(engine)
    
    return OracleAgentMemory(
        engine=engine,
        session_id=session_id,
        agent_id=agent_id
    )


def test_oracle_connection(oracle_config: OracleMemoryConfig) -> bool:
    """Test Oracle database connection"""
    try:
        engine = create_engine(oracle_config.get_connection_string())
        with engine.connect() as conn:
            result = conn.execute(sa.text("SELECT 1 FROM DUAL"))
            return result.fetchone() is not None
    except Exception as e:
        print(f"Connection test failed: {e}")
        return False

print("✓ Factory and utility functions defined!")

---

# Complete Examples and Testing

## Example 1: Mock Memory Store Setup

Demonstrate the Oracle Agent Memory implementation using mock data to avoid requiring an actual Oracle database connection.

In [ ]:
# ============================================================================
# Mock Memory Store for Testing (without Oracle Database)
# ============================================================================

class MockOracleAgentMemory:
    """Mock implementation for testing without actual Oracle database"""
    
    def __init__(self, session_id: str, agent_id: str = "default_agent"):
        self.session_id = session_id
        self.agent_id = agent_id
        self.conversations = []
        self.entities = []
        self.tasks = []
        self.embeddings = []
        self.task_counter = 1
    
    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, str]) -> None:
        """Save conversation context"""
        if "input" in inputs:
            self.conversations.append({
                "type": "human",
                "content": inputs["input"],
                "created_at": datetime.utcnow().isoformat()
            })
        
        if "output" in outputs:
            self.conversations.append({
                "type": "ai",
                "content": outputs["output"],
                "created_at": datetime.utcnow().isoformat()
            })
    
    def add_entity(self, entity_type: str, entity_name: str, entity_value: Dict) -> None:
        """Add entity to memory"""
        self.entities.append({
            "type": entity_type,
            "name": entity_name,
            "value": entity_value,
            "mention_count": 1,
            "last_mentioned": datetime.utcnow().isoformat()
        })
    
    def get_entities(self, entity_type: Optional[str] = None) -> List[Dict]:
        """Get entities from memory"""
        if entity_type:
            return [e for e in self.entities if e["type"] == entity_type]
        return self.entities
    
    def add_task(self, task_description: str, result: Optional[Dict] = None) -> int:
        """Add task to memory"""
        task_id = self.task_counter
        self.task_counter += 1
        self.tasks.append({
            "id": task_id,
            "description": task_description,
            "status": "in_progress",
            "result": result,
            "created_at": datetime.utcnow().isoformat()
        })
        return task_id
    
    def update_task_status(self, task_id: int, status: str, result: Optional[Dict] = None) -> None:
        """Update task status"""
        for task in self.tasks:
            if task["id"] == task_id:
                task["status"] = status
                if result:
                    task["result"] = result
                if status == "completed":
                    task["completed_at"] = datetime.utcnow().isoformat()
                break
    
    def get_conversation_history(self, limit: Optional[int] = None) -> List[Dict]:
        """Get conversation history"""
        history = self.conversations
        if limit:
            return history[-limit:]
        return history
    
    def add_embedding(self, content: str, embedding: List[float], metadata: Optional[Dict] = None) -> None:
        """Add embedding to memory"""
        self.embeddings.append({
            "content": content,
            "embedding": embedding,
            "metadata": metadata or {},
            "created_at": datetime.utcnow().isoformat()
        })
    
    def get_embeddings(self, limit: Optional[int] = None) -> List[Dict]:
        """Get embeddings from memory"""
        if limit:
            return self.embeddings[-limit:]
        return self.embeddings


# Create mock memory instance for demonstrations
print("✓ Mock Memory Store created for testing!")

---

## Example 2: Test Memory Storage Operations

Test all memory storage methods with the mock memory store.

In [ ]:
# Initialize mock memory
mock_memory = MockOracleAgentMemory(
    session_id="chat_session_001",
    agent_id="financial_advisor"
)

print("=" * 70)
print("TESTING MEMORY STORAGE OPERATIONS")
print("=" * 70)

# Test 1: Save conversation context
print("\n1. Saving Conversation Context...")
mock_memory.save_context(
    inputs={"input": "What is the current market trend?"},
    outputs={"output": "Based on recent data, the market shows a bullish trend with tech stocks leading the gains."}
)
mock_memory.save_context(
    inputs={"input": "Which sectors are performing best?"},
    outputs={"output": "Technology, Healthcare, and Energy sectors are showing strong performance this quarter."}
)
print("✓ Conversation context saved")

# Test 2: Add entities
print("\n2. Adding Entities...")
mock_memory.add_entity(
    entity_type="person",
    entity_name="John Smith",
    entity_value={"role": "financial_analyst", "expertise": ["stocks", "bonds"]}
)
mock_memory.add_entity(
    entity_type="organization",
    entity_name="TechCorp",
    entity_value={"sector": "Technology", "market_cap": "500B", "recent_performance": "bullish"}
)
print("✓ Entities added successfully")

# Test 3: Add tasks
print("\n3. Adding Tasks...")
task_id1 = mock_memory.add_task(
    task_description="Analyze quarterly earnings report",
    result={"status": "started"}
)
print(f"✓ Task {task_id1} created: Analyze quarterly earnings")

task_id2 = mock_memory.add_task(
    task_description="Generate portfolio recommendations"
)
print(f"✓ Task {task_id2} created: Generate recommendations")

# Test 4: Update task status
print("\n4. Updating Task Status...")
mock_memory.update_task_status(
    task_id=task_id1,
    status="completed",
    result={"analysis": "Strong performance across all metrics", "recommendation": "Hold"}
)
print(f"✓ Task {task_id1} completed with results")

# Test 5: Add embeddings
print("\n5. Adding Embeddings...")
sample_embeddings = [
    [0.1, 0.2, 0.3, 0.4, 0.5],
    [0.15, 0.25, 0.35, 0.45, 0.55],
    [0.2, 0.3, 0.4, 0.5, 0.6]
]
sample_texts = [
    "Oracle database is powerful for enterprise applications",
    "Vector search enables semantic similarity matching",
    "LangChain provides excellent LLM integration"
]

for text, embedding in zip(sample_texts, sample_embeddings):
    mock_memory.add_embedding(
        content=text,
        embedding=embedding,
        metadata={"source": "documentation", "type": "system_message"}
    )
print("✓ Embeddings added successfully")

print("\n" + "=" * 70)
print("✓ All storage operations completed successfully!")
print("=" * 70)

---

## Example 3: Test Memory Retrieval Operations

Test all memory retrieval methods to extract stored information.

In [ ]:
print("=" * 70)
print("TESTING MEMORY RETRIEVAL OPERATIONS")
print("=" * 70)

# Retrieve conversation history
print("\n1. Conversation History:")
print("-" * 70)
history = mock_memory.get_conversation_history()
for i, msg in enumerate(history, 1):
    msg_type = msg["type"].upper()
    content = msg["content"][:80] + "..." if len(msg["content"]) > 80 else msg["content"]
    print(f"{i}. [{msg_type}] {content}")
print(f"\nTotal messages: {len(history)}")

# Retrieve entities
print("\n2. Stored Entities:")
print("-" * 70)
entities = mock_memory.get_entities()
for entity in entities:
    print(f"  • {entity['type'].upper()}: {entity['name']}")
    print(f"    Value: {entity['value']}")
print(f"\nTotal entities: {len(entities)}")

# Retrieve tasks
print("\n3. Task History:")
print("-" * 70)
for task in mock_memory.tasks:
    print(f"  Task {task['id']}: {task['description']}")
    print(f"    Status: {task['status']}")
    if task['result']:
        print(f"    Result: {task['result']}")
print(f"\nTotal tasks: {len(mock_memory.tasks)}")

# Retrieve embeddings
print("\n4. Stored Embeddings:")
print("-" * 70)
embeddings = mock_memory.get_embeddings()
for i, emb in enumerate(embeddings, 1):
    print(f"{i}. Content: {emb['content'][:60]}...")
    print(f"   Vector dimension: {len(emb['embedding'])}")
    print(f"   Metadata: {emb['metadata']}")
print(f"\nTotal embeddings: {len(embeddings)}")

print("\n" + "=" * 70)
print("✓ All retrieval operations completed successfully!")
print("=" * 70)

---

## Example 4: Visualize Memory State

Create visualizations to understand the memory state and operations.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Oracle Agent Memory Statistics', fontsize=16, fontweight='bold')

# Plot 1: Conversation Distribution
ax1 = axes[0, 0]
message_types = [msg["type"] for msg in mock_memory.get_conversation_history()]
human_count = message_types.count("human")
ai_count = message_types.count("ai")
colors = ['#FF6B6B', '#4ECDC4']
ax1.bar(['Human Messages', 'AI Messages'], [human_count, ai_count], color=colors)
ax1.set_title('Conversation Distribution', fontweight='bold')
ax1.set_ylabel('Count')
for i, v in enumerate([human_count, ai_count]):
    ax1.text(i, v + 0.1, str(v), ha='center', fontweight='bold')

# Plot 2: Task Status Distribution
ax2 = axes[0, 1]
task_statuses = [task["status"] for task in mock_memory.tasks]
status_counts = {}
for status in task_statuses:
    status_counts[status] = status_counts.get(status, 0) + 1
statuses = list(status_counts.keys())
counts = list(status_counts.values())
colors_tasks = ['#95E1D3', '#F38181']
ax2.pie(counts, labels=statuses, autopct='%1.1f%%', colors=colors_tasks, startangle=90)
ax2.set_title('Task Status Distribution', fontweight='bold')

# Plot 3: Entity Types
ax3 = axes[1, 0]
entity_types = [entity["type"] for entity in mock_memory.get_entities()]
entity_type_counts = {}
for etype in entity_types:
    entity_type_counts[etype] = entity_type_counts.get(etype, 0) + 1
types = list(entity_type_counts.keys())
type_counts = list(entity_type_counts.values())
colors_entities = ['#AA96DA', '#FCBAD3']
ax3.barh(types, type_counts, color=colors_entities)
ax3.set_title('Entity Types', fontweight='bold')
ax3.set_xlabel('Count')
for i, v in enumerate(type_counts):
    ax3.text(v + 0.1, i, str(v), va='center', fontweight='bold')

# Plot 4: Memory Content Summary
ax4 = axes[1, 1]
ax4.axis('off')
summary_text = f"""
MEMORY STATE SUMMARY

Session ID: {mock_memory.session_id}
Agent ID: {mock_memory.agent_id}

Conversations: {len(mock_memory.conversations)}
Entities: {len(mock_memory.entities)}
Tasks: {len(mock_memory.tasks)}
Embeddings: {len(mock_memory.embeddings)}

Task Completion:
  • Completed: {len([t for t in mock_memory.tasks if t['status'] == 'completed'])}
  • Pending: {len([t for t in mock_memory.tasks if t['status'] != 'completed'])}
"""
ax4.text(0.1, 0.5, summary_text, fontsize=11, family='monospace',
         verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

print("✓ Memory state visualizations displayed!")

---

## Implementation Guide: Using with Real Oracle Database

### Step 1: Configure Oracle Connection

To connect to a real Oracle database, update your configuration:

```python
# For Oracle Cloud (recommended)
oracle_config = OracleMemoryConfig(
    username="your_username",
    password="your_password",
    host="your_oracle_host",
    port=1521,
    service_name="your_service_name"
)

# OR for traditional Oracle setup
oracle_config = OracleMemoryConfig(
    username="your_username",
    password="your_password",
    host="localhost",
    port=1521,
    sid="orcl"
)
```

### Step 2: Create Memory Instance

```python
# Create and initialize the memory store
memory = create_oracle_agent_memory(
    oracle_config=oracle_config,
    session_id="my_session_001",
    agent_id="my_agent",
    create_tables=True  # Creates tables if they don't exist
)
```

### Step 3: Use with LangChain Agents

```python
from langchain.llms import OpenAI
from langchain.chains import ConversationChain

llm = OpenAI(temperature=0.7, api_key="your_openai_key")

# Create conversation chain with Oracle memory
conversation = ConversationChain(
    llm=llm,
    memory=OracleLangChainMemoryBuilder.create_conversation_memory(
        oracle_config=oracle_config,
        session_id="conversation_001",
        agent_id="chat_bot"
    ),
    verbose=True
)

# Use the conversation
response = conversation.run("How are you today?")
```

### Step 4: Advanced Memory Operations

```python
# Add entities for context
memory.add_entity(
    entity_type="person",
    entity_name="John Doe",
    entity_value={"role": "analyst", "department": "finance"}
)

# Add tasks and track progress
task_id = memory.add_task("Generate quarterly report")
# ... do work ...
memory.update_task_status(task_id, "completed", result={"report": "Q1_2024"})

# Add embeddings for semantic search
memory.add_embedding(
    content="Important business document",
    embedding=[0.1, 0.2, 0.3, ...],
    metadata={"type": "document", "source": "external"}
)
```

---

## Summary and Key Features

### Core Features Implemented

| Feature | Description |
|---------|-------------|
| **Conversation Memory** | Persistent storage of chat history using Oracle database |
| **Entity Tracking** | Extract and track entities mentioned in conversations |
| **Task Management** | Record and monitor agent task execution |
| **Vector Embeddings** | Store embeddings for semantic search capabilities |
| **LangChain Integration** | Full compatibility with LangChain memory and chains |
| **Connection Pooling** | Efficient database connection management |
| **Metadata Support** | Flexible JSON metadata storage for extensibility |

### Database Schema

The implementation uses four main tables:

1. **agent_conversation_memory** - Stores all conversation messages
2. **agent_entity_memory** - Tracks extracted entities and facts
3. **agent_tasks** - Records task execution history
4. **agent_vector_embeddings** - Stores vector embeddings for semantic search

### Key Classes

- `OracleMemoryConfig` - Configuration management
- `OracleAgentMemory` - Main memory store implementation
- `OracleSQLChatMessageHistory` - LangChain chat history integration
- `OracleSQLAgentBuilder` - SQL agent builder utilities
- `MockOracleAgentMemory` - Testing without needing Oracle

### Next Steps

To use this in production:

1. Set up an Oracle database instance (Oracle Cloud, on-premise, or Docker)
2. Install required packages: `sqlalchemy`, `oracledb`, `langchain`
3. Configure your Oracle credentials
4. Initialize the memory store with your configuration
5. Integrate with your LangChain agents and chains

### Additional Resources

- [Oracle Database Documentation](https://docs.oracle.com/en/database/)
- [LangChain Memory Documentation](https://python.langchain.com/docs/modules/memory/)
- [SQLAlchemy + Oracle Setup](https://docs.sqlalchemy.org/en/20/dialects/oracle/)